# Part 1

### 1. Introduction
Netflix manages hundreds of millions of digital assets to deliver high-quality video content to hundreds of millions of users worldwide. These assets include not only video files but also various types of content such as subtitles, audio tracks, thumbnails, images, and metadata.

The asset management system at Netflix serves a role beyond simply storing these assets. Once assets are created, they require continuous updates and reprocessing for various reasons, including adding new versions, modifying metadata, applying new policies or format changes, and improving overall quality [1].

This report explains the structure and key technologies of Netflix's data reprocessing pipeline, which has been developed to automatically handle the reprocessing of such a vast and diverse set of digital assets.

### 2. Reasons for Data Reprocessing
Netflix's digital assets must meet a variety of requirements depending on the country, language, and user experience where the content is provided. However, over time, there are several reasons why existing assets need to be reprocessed:
* **Adding new versions**: When new audio tracks, subtitles, thumbnails, or other components are added to existing video content.
* **Metadata correction**: To fix incorrect information or add new metadata fields.
* **Policy updates**: To comply with new formatting standards, compression requirements, or security policies.
* **Error correction**: To fix errors identified in previous processing stages.
* **Service improvement**: To enhance asset quality for better user experiences and personalized services.  

Given the massive scale of assets managed by Netflix, manually handling these updates is impractical. Therefore, Netflix has developed a fully automated data reprocessing pipeline to efficiently address these needs.

### 3. Data Reprocessing Pipeline Flow
Netflix’s data reprocessing pipeline follows a structured, step-by-step flow to process large volumes of asset data efficiently and reliably. This process can be divided into four main stages: Data Extraction, Event Creation and Transmission, Data Processing, and Error Handling, as shown in Figure 1.

<div align="center">
    <img src="image1.png" alt="Figure 1: Data Reprocessing Pipeline" width="700">
</div>
$$Figure 1: Data Reprocessing Pipeline$$

#### 3.1 Data Extraction
The first step in data reprocessing is extracting asset metadata. Netflix begins the reprocessing workflow by reading the necessary asset metadata stored in the Cassandra database. Cassandra is a distributed NoSQL database designed to store and process massive amounts of data quickly and reliably, making it well-suited for managing Netflix’s vast collection of digital assets [2].

Netflix organizes asset data by type (e.g., VIDEO, TEXT) and creation time to improve the efficiency of data access. The data is structured to support pagination, allowing selective retrieval of only the necessary data portions. To enable this, assets are sorted using a unique identifier called TimeUUID, which is a time-based UUID that contains the timestamp of its creation. TimeUUID makes it easier to sort and access large datasets in chronological order [3]. Pagination refers to a technique of dividing a large dataset into smaller, manageable segments and retrieving them sequentially rather than all at once. This approach helps minimize the load on the system and ensures stable processing, even when dealing with massive datasets [4].

Additionally, when fast searching and filtering of asset data are required, Netflix utilizes Elasticsearch. Elasticsearch is a distributed search engine optimized for quick querying and analysis of large datasets. It allows complex search queries on asset metadata to be executed rapidly [5]. By leveraging these technologies, Netflix establishes a robust foundation for efficiently exploring and reprocessing its large-scale digital assets when necessary.

#### 3.2 Event Creation and Transmission
Once the data extraction is completed, Netflix generates event requests for each asset ID to initiate the reprocessing workflow. These event requests serve as specific instructions or commands necessary for processing asset data and represent the next step in the reprocessing pipeline.

Depending on the nature of the task and the volume of data, events can be processed in either a synchronous or asynchronous manner.
Synchronous processing is used when tasks need to be executed immediately after data extraction. This method is typically applied for small-scale datasets or tasks requiring rapid handling, such as urgent updates to a limited number of assets, as shown in Figure 2.

<div align="center">
    <img src="image2.png" alt="Figure 1: Data Reprocessing Pipeline" width="500">
</div>
$$Figure 2: Synchronous Data Transmission$$  


In contrast, asynchronous processing is mainly employed when handling large datasets (see Figure 3). In this case, Apache Kafka is used to temporarily store a vast number of event requests, which can later be processed either sequentially or in parallel. Apache Kafka is a distributed message queue system designed to transfer massive amounts of data quickly and reliably, making it an essential tool for managing millions of task requests in large-scale services like Netflix [6].

<div align="center">
    <img src="image3.png" alt="Figure 1: Data Reprocessing Pipeline" width="500">
</div>
$$Figure 3: Asynchronous Data Transmission$$  

Kafka organizes these event requests into units called Kafka Topics, which act as temporary storage spaces for the queued tasks. Each Kafka Topic contains groups of events that are later retrieved by various processors for sequential or parallel processing [7]. This mechanism allows Netflix to handle the reprocessing of millions of assets efficiently and reliably in parallel.

In summary, Netflix can flexibly choose between synchronous and asynchronous processing depending on the size of the data and the task requirements. Particularly through Kafka-based asynchronous processing, Netflix can execute large-scale reprocessing workflows smoothly without impacting live services.


#### 3.3 Data Processing
Once events are generated and transmitted, the core reprocessing operations are performed by various data processors. Each processor automatically executes different types of tasks based on the asset IDs provided in the previous stage.

Key processing tasks include updating asset metadata, reindexing assets in Elasticsearch, deleting assets, and performing related operations. For example, asset metadata may be updated to comply with new policies or formats, or asset information may be reindexed to improve search functionality. Furthermore, tasks such as deleting assets whose licenses have expired or cleaning up associated asset data are also handled within this stage.

When dealing with large volumes of data, processing is typically carried out asynchronously. In such cases, a large number of event requests are fetched from Apache Kafka, and parallel processing is performed using thread pools. By leveraging multi-threaded parallel processing, Netflix can efficiently handle millions of asset records in a timely manner.

Additionally, Netflix dynamically adjusts the number of Kafka consumers and the size of the thread pool to manage processing speed according to operational needs. For instance, if reprocessing starts to put a load on the production environment, Netflix can reduce processing speed. Conversely, if faster processing is needed, Netflix can scale up resources to handle more data simultaneously.

Through this well-designed processing system, Netflix can safely and rapidly perform large-scale operations on diverse asset data while maintaining the stability of live services.

#### 3.4 Error Handling
Netflix has established a systematic and robust error handling system to address various error scenarios that may arise during the large-scale data reprocessing process. Since reprocessing vast amounts of asset data can lead to failures due to data quality issues, system outages, or external service problems, it is essential to have mechanisms in place to handle such failures effectively.

When an error occurs during reprocessing, the system is designed to automatically retry the operation, allowing temporary issues to be resolved without manual intervention. However, if the task continues to fail after multiple retries, the system stores the problematic data in a Dead Letter Queue (DLQ) — a separate storage location that allows for later analysis and either manual or automated reprocessing. By storing failed data in the DLQ, Netflix ensures that no data is lost and that the root cause of each issue can be addressed before safe reprocessing takes place (see Figure 4).

<div align="center">
    <img src="image4.png" alt="Figure 1: Data Reprocessing Pipeline" width="500">
</div>
$$Figure 4: Processing clusters$$  

In addition, Netflix utilizes the Atlas framework to collect and monitor failure metrics generated throughout the reprocessing workflow. Atlas is an internal monitoring and visualization framework developed by Netflix that gathers a wide range of system events, performance data, and error metrics. Through real-time dashboards and visualization tools, Atlas enables engineers and system operators to monitor the system's health and promptly detect any anomalies or failures during the reprocessing process.

Importantly, if data reprocessing is found to impact production services, the system is designed to immediately pause or throttle reprocessing operations to mitigate the effect on live services. This capability provides Netflix with the flexibility to maintain service stability while still being able to perform large-scale reprocessing tasks safely and effectively.

### 4. Conclusion
In conclusion, Netflix has built a highly scalable and automated data reprocessing pipeline to efficiently manage and update its massive collection of digital assets. As Netflix continues to deliver content to hundreds of millions of users worldwide, maintaining up-to-date and high-quality assets — including videos, subtitles, audio, images, and metadata — is essential to ensuring a seamless and personalized user experience.

The pipeline is carefully designed to handle the entire reprocessing workflow, from data extraction to event creation, data processing, and error handling. By leveraging advanced technologies such as Cassandra, Elasticsearch, Apache Kafka, and Atlas, Netflix is able to perform large-scale reprocessing tasks without impacting live services. The combination of synchronous and asynchronous event handling, along with parallel processing through multi-threaded consumers, allows for flexible and efficient data handling that adapts to varying workloads and operational demands.

Furthermore, Netflix’s robust error management system, including automatic retries, the use of Dead Letter Queues (DLQ), and real-time monitoring with Atlas, ensures that any issues arising during reprocessing are addressed promptly and effectively, preventing data loss and maintaining system reliability.

Ultimately, this data reprocessing pipeline enables Netflix to rapidly implement new features, correct data errors, comply with evolving content policies, and improve asset quality, all while ensuring the stability and performance of its global streaming platform. The system’s automation and scalability exemplify Netflix’s commitment to operational excellence and high-quality content delivery in a dynamic and demanding digital environment.

### References
1. Netflix Technology Blog. (2022, December 16). Data reprocessing pipeline in asset management platform @Netflix. Medium. https://netflixtechblog.com/data-reprocessing-pipeline-in-asset-management-platform-netflix-46fe225c35c9

2. Apache Cassandra. (n.d.). Welcome to Apache Cassandra. https://cassandra.apache.org/_/index.html

3. DataStax. (n.d.). CQL reference: TimeUUID functions. https://docs.datastax.com/en/cql-oss/3.3/cql/cql_reference/timeuuid_functions_r.html

4. Wikipedia contributors. (n.d.). Pagination. Wikipedia. https://en.wikipedia.org/wiki/Pagination

5. Elasticsearch. (n.d.). Search & analyze data in real time. https://www.elastic.co/elasticsearch

6. Apache Kafka. (n.d.). Welcome to Apache Kafka. https://kafka.apache.org/

7. Apache Kafka. (n.d.). Introduction to Kafka topics. https://kafka.apache.org/documentation/#intro_topics

# Part 2

### pandas-on-Spark

* Read in the weekly nfl data (csv file available at the project page)

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import pyspark.pandas as ps
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [3]:
df = ps.read_csv("weekly_nfl_data.csv")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/16 17:33:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/16 17:33:07 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


* Check out the first 5 rows of the DataFrame

In [4]:
df.head()

25/03/16 17:33:13 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,player_id,player_name,player_display_name,position,position_group,headshot_url,recent_team,season,week,season_type,opponent_team,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_2pt_conversions,pacr,dakota,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr
0,00-0000003,None,Abdul-Karim al-Jabbar,RB,RB,None,MIA,1999,1,REG,DEN,0,0,0.0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,NaN,0,NaN,NaN,16,60.0,1,0.0,0.0,4.0,6.248771,0,1,1,7.0,0,0.0,0.0,0.0,0.0,0.0,0.292378,0,0.0,0.052632,NaN,NaN,0.0,12.7,13.7
1,00-0000003,None,Abdul-Karim al-Jabbar,RB,RB,None,MIA,1999,2,REG,ARI,0,0,0.0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,NaN,0,NaN,NaN,9,33.0,0,0.0,0.0,1.0,-1.434950,0,3,4,18.0,0,0.0,0.0,0.0,0.0,1.0,0.377009,0,0.0,0.117647,NaN,NaN,0.0,5.1,8.1
2,00-0000003,None,Abdul-Karim al-Jabbar,RB,RB,None,MIA,1999,4,REG,BUF,0,0,0.0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,NaN,0,NaN,NaN,3,2.0,0,0.0,0.0,0.0,-1.539952,0,0,1,0.0,0,0.0,0.0,0.0,0.0,0.0,-0.699578,0,NaN,0.023810,NaN,NaN,0.0,0.2,0.2
3,00-0000003,None,Abdul-Karim al-Jabbar,RB,RB,None,CLE,1999,7,REG,LA,0,0,0.0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,NaN,0,NaN,NaN,6,27.0,0,0.0,0.0,0.0,0.216051,0,2,2,8.0,0,0.0,0.0,0.0,0.0,0.0,-0.228454,0,0.0,0.050000,NaN,NaN,0.0,3.5,5.5
4,00-0000003,None,Abdul-Karim al-Jabbar,RB,RB,None,CLE,1999,8,REG,NO,0,0,0.0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,NaN,0,NaN,NaN,13,39.0,0,0.0,0.0,2.0,-2.972259,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,3.9,3.9


* Report all of the column names

In [5]:
df.columns

Index(['player_id', 'player_name', 'player_display_name', 'position',
       'position_group', 'headshot_url', 'recent_team', 'season', 'week',
       'season_type', 'opponent_team', 'completions', 'attempts',
       'passing_yards', 'passing_tds', 'interceptions', 'sacks', 'sack_yards',
       'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards',
       'passing_yards_after_catch', 'passing_first_downs', 'passing_epa',
       'passing_2pt_conversions', 'pacr', 'dakota', 'carries', 'rushing_yards',
       'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost',
       'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions',
       'receptions', 'targets', 'receiving_yards', 'receiving_tds',
       'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards',
       'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa',
       'receiving_2pt_conversions', 'racr', 'target_share', 'air_yards_share',
       'wopr', 'special_teams_tds', 'fantasy_points

* We want to only look at QB stats for the seasons 2005 to 2023 (inclusive).  
1) Subset the rows of the data to only include the position “QB”, the regular season (“REG”), and season in the range noted above  
2) Subset the columns to only include the player_display_name, season, week, completions, attempts, passing_yards, passing_tds, and interceptions  
3) For each player_display_name and season combination, fine the sum and mean of each of the statistical quantities (the rest of the columns we chose above)  
4) Create two new variables (by season/player combination):  
    * completion_percentage = (sum of completions)/(sum of attempts)
    * td_int_ratio = (sum passing tds)/(sum interceptions)

In [6]:
# Filter the data for the 2005~2003 seasons.
# We can write the code in the same way as using pandas.
df_qb = df[(df["position"] == "QB") & 
           (df["season_type"] == "REG") &
           (df["season"] >= 2005) &
           (df["season"] <= 2023)]

In [7]:
# Select only the necessary columns, just like handling a DataFrame.
df_qb = df_qb[["player_display_name", "season", "week", "completions", "attempts", "passing_yards", "passing_tds", "interceptions"]]

In [8]:
# Use .groupby() to create combinations of player_display_name and season.
# Use .agg() to apply sum and mean functions to the grouped data.
df_qb_stats = df_qb \
                  .groupby(["player_display_name", "season"])  \
                  .agg(["sum", "mean"])
df_qb_stats.head()

week            completions            attempts            passing_yards             passing_tds           interceptions          
                            sum       mean         sum       mean      sum       mean           sum        mean         sum      mean           sum      mean
player_display_name season                                                                                                                                   
Jake Delhomme       2006     99   7.615385         263  20.230769      431  33.153846        2805.0  215.769231          17  1.307692          11.0  0.846154
Jake Plummer        2005    144   9.000000         277  17.312500      456  28.500000        3366.0  210.375000          18  1.125000           7.0  0.437500
Matt Schaub         2006     60  12.000000          18   3.600000       27   5.400000         208.0   41.600000           1  0.200000           2.0  0.400000
Vince Young         2006    143   9.533333         184  12.266667      356  23.733333        2199.0  146.600000          12  0.800000          13.0  0.866667
Kerry Collins       2007     48   8.000000          50   8.333333       82  13.666667         531.0   88.500000           0  0.000000           0.0  0.000000

In [9]:
# Since the structure uses a multi-index, columns should be selected using [(main level), (sub level)].
# First, calculate the 'completion_percentage' value.
df_qb_stats["completion_percentage"] = df_qb_stats[("completions", "sum")] / df_qb_stats[("attempts", "sum")]
df_qb_stats.head()

week            completions            attempts            passing_yards             passing_tds           interceptions           completion_percentage
                            sum       mean         sum       mean      sum       mean           sum        mean         sum      mean           sum      mean                      
player_display_name season                                                                                                                                                         
Jake Delhomme       2006     99   7.615385         263  20.230769      431  33.153846        2805.0  215.769231          17  1.307692          11.0  0.846154              0.610209
Jake Plummer        2005    144   9.000000         277  17.312500      456  28.500000        3366.0  210.375000          18  1.125000           7.0  0.437500              0.607456
Matt Schaub         2006     60  12.000000          18   3.600000       27   5.400000         208.0   41.600000           1  0.200000           2.0  0.400000              0.666667
Vince Young         2006    143   9.533333         184  12.266667      356  23.733333        2199.0  146.600000          12  0.800000          13.0  0.866667              0.516854
Kerry Collins       2007     48   8.000000          50   8.333333       82  13.666667         531.0   88.500000           0  0.000000           0.0  0.000000              0.609756

In [10]:
# Next, calculate the 'td_int_ratio' value 
df_qb_stats["td_int_ratio"] = df_qb_stats[("passing_tds", "sum")] / df_qb_stats[("interceptions", "sum")]
df_qb_stats.head()

week            completions            attempts            passing_yards             passing_tds           interceptions           completion_percentage td_int_ratio
                            sum       mean         sum       mean      sum       mean           sum        mean         sum      mean           sum      mean                                   
player_display_name season                                                                                                                                                                      
Jake Delhomme       2006     99   7.615385         263  20.230769      431  33.153846        2805.0  215.769231          17  1.307692          11.0  0.846154              0.610209     1.545455
Jake Plummer        2005    144   9.000000         277  17.312500      456  28.500000        3366.0  210.375000          18  1.125000           7.0  0.437500              0.607456     2.571429
Matt Schaub         2006     60  12.000000          18   3.600000       27   5.400000         208.0   41.600000           1  0.200000           2.0  0.400000              0.666667     0.500000
Vince Young         2006    143   9.533333         184  12.266667      356  23.733333        2199.0  146.600000          12  0.800000          13.0  0.866667              0.516854     0.923077
Kerry Collins       2007     48   8.000000          50   8.333333       82  13.666667         531.0   88.500000           0  0.000000           0.0  0.000000              0.609756          NaN

* Save the result of above as an object. With that object
1) Subset the rows to only include player/season combinations wher ethe sum of attempts is at least 50.
2) Sort the rows descending by completion_percentage and report the first 40 values!
3) Sort the rows descending by td_int_ratio and report the first 40 values!

In [11]:
#  Filter the data to include only records where the sum of attempts is greater than or equal to 50.
df_qb_filtered = df_qb_stats[df_qb_stats[("attempts", "sum")] >= 50]
df_qb_filtered.head()

week            completions            attempts            passing_yards             passing_tds           interceptions           completion_percentage td_int_ratio
                            sum       mean         sum       mean      sum       mean           sum        mean         sum      mean           sum      mean                                   
player_display_name season                                                                                                                                                                      
Jake Delhomme       2006     99   7.615385         263  20.230769      431  33.153846        2805.0  215.769231          17  1.307692          11.0  0.846154              0.610209     1.545455
Jake Plummer        2005    144   9.000000         277  17.312500      456  28.500000        3366.0  210.375000          18  1.125000           7.0  0.437500              0.607456     2.571429
Vince Young         2006    143   9.533333         184  12.266667      356  23.733333        2199.0  146.600000          12  0.800000          13.0  0.866667              0.516854     0.923077
Kerry Collins       2007     48   8.000000          50   8.333333       82  13.666667         531.0   88.500000           0  0.000000           0.0  0.000000              0.609756          NaN
Charlie Batch       2006     71  10.142857          30   4.285714       52   7.428571         477.0   68.142857           5  0.714286           0.0  0.000000              0.576923          inf

In [12]:
# Use sort_values() to sort the DataFrame based on the values of a specific column (completion_percentage).
# Set ascending=False to sort in descending order.
# Use head(40) to display the top 40 rows.
df_qb_filtered.sort_values(by="completion_percentage", ascending=False).head(40)

week            completions            attempts            passing_yards             passing_tds           interceptions           completion_percentage td_int_ratio
                            sum       mean         sum       mean      sum       mean           sum        mean         sum      mean           sum      mean                                   
player_display_name season                                                                                                                                                                      
C.J. Beathard       2023     65  10.833333          40   6.666667       53   8.833333         349.0   58.166667           1  0.166667           0.0  0.000000              0.754717          inf
Colt McCoy          2021     62   8.857143          74  10.571429       99  14.142857         740.0  105.714286           3  0.428571           1.0  0.142857              0.747475     3.000000
Matt Schaub         2019     52  10.400000          50  10.000000       67  13.400000         580.0  116.000000           3  0.600000           1.0  0.200000              0.746269     3.000000
Drew Brees          2018    130   8.666667         364  24.266667      489  32.600000        3992.0  266.133333          32  2.133333           5.0  0.333333              0.744376     6.400000
                    2019    119  10.818182         281  25.545455      378  34.363636        2979.0  270.818182          27  2.454545           4.0  0.363636              0.743386     6.750000
Mason Rudolph       2023     66  16.500000          55  13.750000       74  18.500000         719.0  179.750000           3  0.750000           0.0  0.000000              0.743243          inf
Taysom Hill         2020    147   9.187500          88   5.500000      121   7.562500         928.0   58.000000           4  0.250000           2.0  0.125000              0.727273     2.000000
Nick Foles          2018     51  10.200000         141  28.200000      195  39.000000        1413.0  282.600000           7  1.400000           4.0  0.800000              0.723077     1.750000
Drew Brees          2017    148   9.250000         386  24.125000      536  33.500000        4334.0  270.875000          23  1.437500           8.0  0.500000              0.720149     2.875000
Sam Bradford        2016    146   9.733333         395  26.333333      552  36.800000        3877.0  258.466667          20  1.333333           5.0  0.333333              0.715580     4.000000
Drew Brees          2011    142   8.875000         471  29.437500      660  41.250000        5535.0  345.937500          46  2.875000          14.0  0.875000              0.713636     3.285714
Colt McCoy          2014     57  11.400000          91  18.200000      128  25.600000        1057.0  211.400000           4  0.800000           3.0  0.600000              0.710938     1.333333
Aaron Rodgers       2020    148   9.250000         372  23.250000      526  32.875000        4299.0  268.687500          48  3.000000           5.0  0.312500              0.707224     9.600000
Bailey Zappe        2022     22   5.500000          65  16.250000       92  23.000000         781.0  195.250000           5  1.250000           3.0  0.750000              0.706522     1.666667
Drew Brees          2009    131   8.733333         363  24.200000      514  34.266667        4388.0  292.533333          34  2.266667          11.0  0.733333              0.706226     3.090909
                    2020     97   8.083333         275  22.916667      390  32.500000        2942.0  245.166667          24  2.000000           6.0  0.500000              0.705128     4.000000
Joe Burrow          2021    143   8.937500         366  22.875000      520  32.500000        4611.0  288.187500          34  2.125000          14.0  0.875000              0.703846     2.428571
Derek Carr          2019    147   9.187500         361  22.562500      513  32.062500        4054.0  253.375000          21  1.312500           8.0  0.500000          

In [13]:
# Use sort_values() to sort the DataFrame based on the values of a specific column (td_int_ratio).
# Set ascending=False to sort in descending order.
# Use head(40) to display the top 40 rows.
df_qb_filtered.sort_values(by="td_int_ratio", ascending=False).head(40)

week            completions            attempts            passing_yards             passing_tds           interceptions           completion_percentage td_int_ratio
                            sum       mean         sum       mean      sum       mean           sum        mean         sum      mean           sum      mean                                   
player_display_name season                                                                                                                                                                      
Charlie Batch       2006     71  10.142857          30   4.285714       52   7.428571         477.0   68.142857           5  0.714286           0.0  0.000000              0.576923          inf
Matt Schaub         2005     65   9.285714          33   4.714286       64   9.142857         495.0   70.714286           4  0.571429           0.0  0.000000              0.515625          inf
Todd Collins        2007     62  15.500000          67  16.750000      105  26.250000         888.0  222.000000           5  1.250000           0.0  0.000000              0.638095          inf
Troy Smith          2007     62  15.500000          40  10.000000       76  19.000000         452.0  113.000000           2  0.500000           0.0  0.000000              0.526316          inf
Jake Locker         2011     51  10.200000          34   6.800000       66  13.200000         542.0  108.400000           4  0.800000           0.0  0.000000              0.515152          inf
Brian Hoyer         2016     27   4.500000         134  22.333333      200  33.333333        1445.0  240.833333           6  1.000000           0.0  0.000000              0.670000          inf
Nick Foles          2016     17   8.500000          36  18.000000       55  27.500000         410.0  205.000000           3  1.500000           0.0  0.000000              0.654545          inf
Derek Anderson      2014     30   6.000000          65  13.000000       97  19.400000         701.0  140.200000           5  1.000000           0.0  0.000000              0.670103          inf
Jimmy Garoppolo     2016     49   8.166667          43   7.166667       63  10.500000         502.0   83.666667           4  0.666667           0.0  0.000000              0.682540          inf
Matt Moore          2019     54   9.000000          59   9.833333       91  15.166667         659.0  109.833333           4  0.666667           0.0  0.000000              0.648352          inf
C.J. Beathard       2020     67  11.166667          66  11.000000      104  17.333333         787.0  131.166667           6  1.000000           0.0  0.000000              0.634615          inf
Andy Dalton         2023      8   4.000000          34  17.000000       58  29.000000         361.0  180.500000           2  1.000000           0.0  0.000000              0.586207          inf
Mason Rudolph       2023     66  16.500000          55  13.750000       74  18.500000         719.0  179.750000           3  0.750000           0.0  0.000000              0.743243          inf
Desmond Ridder      2022     66  16.500000          73  18.250000      115  28.750000         708.0  177.000000           2  0.500000           0.0  0.000000              0.634783          inf
C.J. Beathard       2023     65  10.833333          40   6.666667       53   8.833333         349.0   58.166667           1  0.166667           0.0  0.000000              0.754717          inf
Tom Brady           2016    134  11.166667         291  24.250000      432  36.000000        3554.0  296.166667          28  2.333333           2.0  0.166667              0.673611    14.000000
Nick Foles          2013    129   9.923077         203  15.615385      317  24.384615        2891.0  222.384615          27  2.076923           2.0  0.153846              0.640379    13.500000
Josh McCown         2013     92  11.500000         149  18.625000      224  28.000000        1829.0  228.625000          13  1.625000           1.0  0.125000          

### Spark SQL DataFrame

* Read in the weekly nfl data (csv file available at the project page)

In [14]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import sum, avg, col, try_divide, desc

In [15]:
spark = SparkSession.builder.master('local[*]').appName('NFL_data').getOrCreate()

25/03/16 17:33:23 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [16]:
sql_df = spark.read.load("weekly_nfl_data.csv",
                     format="csv", 
                     sep=",", 
                     inferSchema="true", 
                     header="true")

* Check out the first 5 rows of the DataFrame

In [17]:
# Spark SQL uses .show() instead of .head().
# The result is displayed as a table in text format. 
# Since the NFL dataset contains many columns, the output may be too wide and appear broken on the screen.
sql_df.show(5)

+----------+-----------+--------------------+--------+--------------+------------+-----------+------+----+-----------+-------------+-----------+--------+-------------+-----------+-------------+-----+----------+------------+-----------------+-----------------+-------------------------+-------------------+-----------+-----------------------+----+------+-------+-------------+-----------+---------------+--------------------+-------------------+-----------+-----------------------+----------+-------+---------------+-------------+-----------------+----------------------+-------------------+---------------------------+---------------------+-------------+-------------------------+----+------------+---------------+----+-----------------+--------------+------------------+
| player_id|player_name| player_display_name|position|position_group|headshot_url|recent_team|season|week|season_type|opponent_team|completions|attempts|passing_yards|passing_tds|interceptions|sacks|sack_yards|sack_fumbles|sack_

* Report all of the column names

In [18]:
# To display column names, use .columns just like pandas-on-Spark.
sql_df.columns

['player_id',
 'player_name',
 'player_display_name',
 'position',
 'position_group',
 'headshot_url',
 'recent_team',
 'season',
 'week',
 'season_type',
 'opponent_team',
 'completions',
 'attempts',
 'passing_yards',
 'passing_tds',
 'interceptions',
 'sacks',
 'sack_yards',
 'sack_fumbles',
 'sack_fumbles_lost',
 'passing_air_yards',
 'passing_yards_after_catch',
 'passing_first_downs',
 'passing_epa',
 'passing_2pt_conversions',
 'pacr',
 'dakota',
 'carries',
 'rushing_yards',
 'rushing_tds',
 'rushing_fumbles',
 'rushing_fumbles_lost',
 'rushing_first_downs',
 'rushing_epa',
 'rushing_2pt_conversions',
 'receptions',
 'targets',
 'receiving_yards',
 'receiving_tds',
 'receiving_fumbles',
 'receiving_fumbles_lost',
 'receiving_air_yards',
 'receiving_yards_after_catch',
 'receiving_first_downs',
 'receiving_epa',
 'receiving_2pt_conversions',
 'racr',
 'target_share',
 'air_yards_share',
 'wopr',
 'special_teams_tds',
 'fantasy_points',
 'fantasy_points_ppr']

* We want to only look at QB stats for the seasons 2005 to 2023 (inclusive).  
1) Subset the rows of the data to only include the position “QB”, the regular season (“REG”), and season in the range noted above  
2) Subset the columns to only include the player_display_name, season, week, completions, attempts, passing_yards, passing_tds, and interceptions  
3) For each player_display_name and season combination, fine the sum and mean of each of the statistical quantities (the rest of the columns we chose above)  
4) Create two new variables (by season/player combination):  
    * completion_percentage = (sum of completions)/(sum of attempts)
    * td_int_ratio = (sum passing tds)/(sum interceptions)

In [19]:
# Spark SQL allows filtering using SQL-style conditions as follows.
sql_df_qb = sql_df.filter("position == 'QB' AND season_type == 'REG' AND season >= 2005 AND season <= 2023")

In [20]:
# To select specific columns, use .select(column_name).
sql_df_qb = sql_df_qb.select("player_display_name", "season", "week", "completions", "attempts", "passing_yards", "passing_tds", "interceptions")

In [21]:
stats = ["completions", "attempts", "passing_yards", "passing_tds", "interceptions"]

sql_df_qb_stats = sql_df_qb.groupBy("player_display_name", "season").agg(   \
                              *[sum(col(stat)).alias(f"{stat}_sum") for stat in stats],  \
                              *[avg(col(stat)).alias(f"{stat}_mean") for stat in stats])
sql_df_qb_stats.show(5)

+-------------------+------+---------------+------------+-----------------+---------------+-----------------+------------------+------------------+------------------+------------------+------------------+
|player_display_name|season|completions_sum|attempts_sum|passing_yards_sum|passing_tds_sum|interceptions_sum|  completions_mean|     attempts_mean|passing_yards_mean|  passing_tds_mean|interceptions_mean|
+-------------------+------+---------------+------------+-----------------+---------------+-----------------+------------------+------------------+------------------+------------------+------------------+
|      Jake Delhomme|  2006|            263|         431|           2805.0|             17|             11.0| 20.23076923076923| 33.15384615384615|215.76923076923077|1.3076923076923077|0.8461538461538461|
|       Jake Plummer|  2005|            277|         456|           3366.0|             18|              7.0|           17.3125|              28.5|           210.375|             1

In [22]:
sql_df_qb_stats = sql_df_qb_stats.withColumn("completion_percentage",   \
                                              try_divide(col("completions_sum"), col("attempts_sum")))

sql_df_qb_stats.select("completion_percentage").show(5)

+---------------------+
|completion_percentage|
+---------------------+
|   0.6102088167053364|
|   0.6074561403508771|
|   0.6666666666666666|
|   0.5168539325842697|
|   0.6097560975609756|
+---------------------+
only showing top 5 rows



In [23]:
# Use try_divide() to divide the value of the 'passing_tds_sum' column by the 'interceptions_sum' column
# and store the result in a new column named 'td_int_ratio'.
# This function returns null instead of an error if the denominator is 0.
sql_df_qb_stats = sql_df_qb_stats.withColumn("td_int_ratio",    \
                                             try_divide(col("passing_tds_sum"), col("interceptions_sum")))

sql_df_qb_stats.select("td_int_ratio").show(5)

+------------------+
|      td_int_ratio|
+------------------+
|1.5454545454545454|
|2.5714285714285716|
|               0.5|
|0.9230769230769231|
|              NULL|
+------------------+
only showing top 5 rows



* Save the result of above as an object. With that object
1) Subset the rows to only include player/season combinations wher ethe sum of attempts is at least 50.
2) Sort the rows descending by completion_percentage and report the first 40 values!
3) Sort the rows descending by td_int_ratio and report the first 40 values!

In [24]:
# Use .filter() to filter rows where the value of 'attempts_sum' is greater than or equal to 50.
sql_df_qb_filtered = sql_df_qb_stats.filter(col("attempts_sum") >= 50)
sql_df_qb_filtered.show(5)

+-------------------+------+---------------+------------+-----------------+---------------+-----------------+------------------+------------------+------------------+------------------+------------------+---------------------+------------------+
|player_display_name|season|completions_sum|attempts_sum|passing_yards_sum|passing_tds_sum|interceptions_sum|  completions_mean|     attempts_mean|passing_yards_mean|  passing_tds_mean|interceptions_mean|completion_percentage|      td_int_ratio|
+-------------------+------+---------------+------------+-----------------+---------------+-----------------+------------------+------------------+------------------+------------------+------------------+---------------------+------------------+
|      Jake Delhomme|  2006|            263|         431|           2805.0|             17|             11.0| 20.23076923076923| 33.15384615384615|215.76923076923077|1.3076923076923077|0.8461538461538461|   0.6102088167053364|1.5454545454545454|
|       Jake Plu

In [25]:
# Use .orderBy() to sort the DataFrame based on the values of a specific column (completion_percentage).
# Use desc() to specify descending order.
# Use .show(40) to display the top 40 rows.
sql_df_qb_filtered.orderBy(desc("completion_percentage")).show(40)

+-------------------+------+---------------+------------+-----------------+---------------+-----------------+------------------+------------------+------------------+-------------------+-------------------+---------------------+------------------+
|player_display_name|season|completions_sum|attempts_sum|passing_yards_sum|passing_tds_sum|interceptions_sum|  completions_mean|     attempts_mean|passing_yards_mean|   passing_tds_mean| interceptions_mean|completion_percentage|      td_int_ratio|
+-------------------+------+---------------+------------+-----------------+---------------+-----------------+------------------+------------------+------------------+-------------------+-------------------+---------------------+------------------+
|      C.J. Beathard|  2023|             40|          53|            349.0|              1|              0.0| 6.666666666666667| 8.833333333333334|58.166666666666664|0.16666666666666666|                0.0|   0.7547169811320755|              NULL|
|       

In [26]:
# Use .orderBy() to sort the DataFrame based on the values of a specific column (td_int_ratio).
# Use desc() to specify descending order.
# Use .show(40) to display the top 40 rows. 
sql_df_qb_filtered.orderBy(desc("td_int_ratio")).show(40)

+-------------------+------+---------------+------------+-----------------+---------------+-----------------+------------------+------------------+------------------+------------------+-------------------+---------------------+-----------------+
|player_display_name|season|completions_sum|attempts_sum|passing_yards_sum|passing_tds_sum|interceptions_sum|  completions_mean|     attempts_mean|passing_yards_mean|  passing_tds_mean| interceptions_mean|completion_percentage|     td_int_ratio|
+-------------------+------+---------------+------------+-----------------+---------------+-----------------+------------------+------------------+------------------+------------------+-------------------+---------------------+-----------------+
|          Tom Brady|  2016|            291|         432|           3554.0|             28|              2.0|             24.25|              36.0| 296.1666666666667|2.3333333333333335|0.16666666666666666|   0.6736111111111112|             14.0|
|         Nick F